### Necessary Imports and Installs

In [3]:
!pip install opendatasets

In [4]:
#!pip install cartopy

In [5]:
!rm -rf GLC
!git clone https://github.com/maximiliense/GLC

Cloning into 'GLC'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 383 (delta 119), reused 170 (delta 63), pack-reused 155
Receiving objects: 100% (383/383), 10.57 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [6]:
!rm -rf GLC
!git clone https://github.com/maximiliense/GLC

Cloning into 'GLC'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 383 (delta 119), reused 170 (delta 63), pack-reused 155
Receiving objects: 100% (383/383), 10.57 MiB | 25.17 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Conv2D, Dense, AveragePooling2D, \
BatchNormalization, Normalization, Dropout, Flatten, Lambda, Input, Activation
from tensorflow.keras import Model
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
#import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import scale
import time
from collections import defaultdict
import math
import copy
import opendatasets as od

%pylab inline --no-import-all
from pathlib import Path
import pandas as pd
import sys

Populating the interactive namespace from numpy and matplotlib


# Dataset

## Download Dataset

{"username":"nathaliemh","key":"cba80d1f619e96b238e4a95aa3017836"}

In [8]:
data = od.download("https://www.kaggle.com/competitions/geolifeclef-2022-lifeclef-2022-fgvc9")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nathaliemh
Your Kaggle Key: ········


  0%|          | 2.00M/57.6G [00:00<52:06, 19.8MB/s]

100%|██████████| 57.6G/57.6G [29:45<00:00, 34.6MB/s]  



Extracting archive ./geolifeclef-2022-lifeclef-2022-fgvc9/geolifeclef-2022-lifeclef-2022-fgvc9.zip to ./geolifeclef-2022-lifeclef-2022-fgvc9


## Load Dataset from file

In [2]:
# Change this path to adapt to where you downloaded the data
DATA_PATH = Path("./geolifeclef-2022-lifeclef-2022-fgvc9/")

In [3]:
import time
hours = 4
#time.sleep(60*60*hours)

In [4]:
ls -L $DATA_PATH

metadata/              patches-us/            rasters/
observations/          patches_sample/        sample_submission.csv
patches-fr/            pre-extracted/


In [5]:
ls $DATA_PATH/observations

observations_fr_test.csv   observations_us_test.csv
observations_fr_train.csv  observations_us_train.csv


In [6]:
### Training Dataset ###
# let's load the data from file
df_obs_fr = pd.read_csv(DATA_PATH / "observations" / "observations_fr_train.csv", sep=";", index_col="observation_id")
df_obs_us = pd.read_csv(DATA_PATH / "observations" / "observations_us_train.csv", sep=";", index_col="observation_id")

df_obs = pd.concat((df_obs_fr, df_obs_us))

print("Number of observations for training: {}".format(len(df_obs)))

# let's have a look at the data
df_obs.head()

Number of observations for training: 1627475


,latitude,longitude,species_id,subset
observation_id,,,,
10561949,45.705116,1.424622,241,train
10131188,45.146973,6.416794,101,train
10799362,46.783695,-2.072855,700,train
10392536,48.604866,-2.825003,1456,train
10335049,48.815567,-0.161431,157,train


In [7]:
### Test Dataset ###
df_obs_fr_test = pd.read_csv(DATA_PATH / "observations" / "observations_fr_test.csv", sep=";", index_col="observation_id")
df_obs_us_test = pd.read_csv(DATA_PATH / "observations" / "observations_us_test.csv", sep=";", index_col="observation_id")

df_obs_test = pd.concat((df_obs_fr_test, df_obs_us_test))

print("Number of observations for testing: {}".format(len(df_obs_test)))

df_obs_test.head()

Number of observations for testing: 36421


,latitude,longitude
observation_id,,
10782781,43.601788,6.940195
10364138,46.241711,0.683586
10692017,45.181095,1.533459
10222322,46.938450,5.298678
10241950,45.017433,0.960736


In [8]:
ls $DATA_PATH/patches-fr/00

00/ 05/ 10/ 15/ 20/ 25/ 30/ 35/ 40/ 45/ 50/ 55/ 60/ 65/ 70/ 75/ 80/ 85/ 90/ 95/
01/ 06/ 11/ 16/ 21/ 26/ 31/ 36/ 41/ 46/ 51/ 56/ 61/ 66/ 71/ 76/ 81/ 86/ 91/ 96/
02/ 07/ 12/ 17/ 22/ 27/ 32/ 37/ 42/ 47/ 52/ 57/ 62/ 67/ 72/ 77/ 82/ 87/ 92/ 97/
03/ 08/ 13/ 18/ 23/ 28/ 33/ 38/ 43/ 48/ 53/ 58/ 63/ 68/ 73/ 78/ 83/ 88/ 93/ 98/
04/ 09/ 14/ 19/ 24/ 29/ 34/ 39/ 44/ 49/ 54/ 59/ 64/ 69/ 74/ 79/ 84/ 89/ 94/ 99/


In [9]:
df_suggested_landcover_alignment = pd.read_csv(DATA_PATH / "metadata" / "landcover_suggested_alignment.csv", sep=";")
df_suggested_landcover_alignment.head()

,landcover_code,suggested_landcover_code,suggested_landcover_label
0,0,0,Missing Data
1,1,11,Cultivated Crops
2,2,11,Cultivated Crops
3,3,6,Broad-leaved Forest
4,4,7,Coniferous Forest


In [10]:
from GLC.data_loading.common import load_patch

patch = load_patch(10171444, DATA_PATH)

print("Number of data sources: {}".format(len(patch)))
print("Arrays shape: {}".format([p.shape for p in patch]))
print("Data types: {}".format([p.dtype for p in patch]))

Number of data sources: 4
Arrays shape: [(256, 256, 3), (256, 256), (256, 256), (256, 256)]
Data types: [dtype('uint8'), dtype('uint8'), dtype('int16'), dtype('uint8')]


In [11]:
landcover_mapping = df_suggested_landcover_alignment["suggested_landcover_code"].values
#patch = load_patch(10171444, DATA_PATH, landcover_mapping=landcover_mapping)

In [19]:
"""from GLC.plotting import visualize_observation_patch

# Extracts land cover labels
landcover_labels = df_suggested_landcover_alignment[["suggested_landcover_code", "suggested_landcover_label"]].drop_duplicates().sort_values("suggested_landcover_code")["suggested_landcover_label"].values

visualize_observation_patch(patch, observation_data=df_obs.loc[10561900], landcover_labels=landcover_labels)
"""

'from GLC.plotting import visualize_observation_patch\n\n# Extracts land cover labels\nlandcover_labels = df_suggested_landcover_alignment[["suggested_landcover_code", "suggested_landcover_label"]].drop_duplicates().sort_values("suggested_landcover_code")["suggested_landcover_label"].values\n\nvisualize_observation_patch(patch, observation_data=df_obs.loc[10561900], landcover_labels=landcover_labels)\n'

In [20]:
"""patch = load_patch(22068100, DATA_PATH, landcover_mapping=landcover_mapping)

visualize_observation_patch(patch, observation_data=df_obs.loc[22068100], landcover_labels=landcover_labels)
"""

'patch = load_patch(22068100, DATA_PATH, landcover_mapping=landcover_mapping)\n\nvisualize_observation_patch(patch, observation_data=df_obs.loc[22068100], landcover_labels=landcover_labels)\n'

In [21]:
#np.shape(patch[0])

### Train/Val Split Labels
Retrieve the train/val split provided.

In [12]:
obs_id_train = df_obs.index[df_obs["subset"] == "train"].values
obs_id_val = df_obs.index[df_obs["subset"] == "val"].values

y_train = df_obs.loc[obs_id_train]["species_id"].values
y_val = df_obs.loc[obs_id_val]["species_id"].values

n_val = len(obs_id_val)
print("Training set size: {} ({:.1%} of train observations)".format(len(y_train), len(y_train) / len(df_obs)))
print("Validation set size: {} ({:.1%} of train observations)".format(n_val, n_val / len(df_obs)))

Training set size: 1587395 (97.5% of train observations)
Validation set size: 40080 (2.5% of train observations)


### Load patches

In [13]:
np.shape(y_train)

(1587395,)

In [14]:
# load training dataset samples
# factor = 1 means load full training dataset
# factor = 100 means load 1/100 of the full dataset
factor = 100

X_train = list() #np.array((np.shape(y_train), 256, 256, 3))
for obs_id in obs_id_train:
    patch = load_patch(obs_id, DATA_PATH, landcover_mapping=landcover_mapping)
    X_train.append(patch[0])
    
    percent_progress = len(X_train)/(len(y_train)/factor) * 100
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
    sys.stdout.flush()
        
    if len(X_train) >= (len(y_train)/factor):
        break
print()
    
X_train = np.array(X_train)
print("sample array shape: ", np.shape(X_train))
print("label array shape: ", np.shape(y_train))

[====================] 100%
sample array shape:  (15874, 256, 256, 3)
label array shape:  (1587395,)


In [15]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train[:len(X_train)]))

In [16]:
train_ds = train_ds.batch(64)

In [17]:
len(np.unique(y_train[:len(X_train)]))

2254

In [28]:
# load validation dataset samples
factor = 1

X_val = list() #np.array((np.shape(y_train), 256, 256, 3))
for obs_id in obs_id_val:
    patch = load_patch(obs_id, DATA_PATH, landcover_mapping=landcover_mapping)
    X_val.append(patch[0])
    
    percent_progress = len(X_val)/(len(y_val)/factor) * 100
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*int(percent_progress/5), percent_progress))
    sys.stdout.flush()
    
    if len(X_val) >= (len(y_val)/factor):
        break

print()
    
X_val = np.array(X_val)
print("sample array shape: ", np.shape(X_val))
print("label array shape: ", np.shape(y_val))

[====================] 100%
sample array shape:  (40080, 256, 256, 3)
label array shape:  (40080,)


In [29]:
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val[:len(X_val)]))
val_ds = val_ds.batch(64)

## Preprocess

# First Simple Neural Network
Let's create a first neural network as a baseline to see how it performs.

In [18]:
# returns a 10 layer ReLU model of width 2
def simple_model(input_shape):
    model = tf.keras.models.Sequential()
    
    # 1. Preprocessing
    # rescale inputs
    model.add(tf.keras.layers.Rescaling(1./255))

    # 2. Convolutional Layers
    model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=input_shape, padding='same'))
    #model.add(AveragePooling2D())

    model.add(Conv2D(64, kernel_size=5, activation='relu', padding='same'))
    #model.add(AveragePooling2D())
    
    model.add(Conv2D(128, kernel_size=5, activation='relu', padding='same'))
    
    # from convolutional layers to dense layers
    model.add(tf.keras.layers.Flatten())
    
    # 3. Dense Layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    
    # 4. Output Layer
    model.add(Dense(4908, activation='softmax'))
    
    # compire the model
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    
    return model

In [19]:
# create the network
model = simple_model((256, 256, 3))

In [20]:
np.max(y_train[:len(X_train)])

4907

In [21]:
for y in y_train[:len(X_train)]:
    if y > np.max(y_train[:len(X_train)]):
        print(y)

In [22]:
np.min(y_train[:len(X_train)])

0

Train the network.

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.001, patience=5, 
                                              verbose=0, mode='auto', baseline=None, restore_best_weights=True)


In [ ]:
history = model.fit(train_ds, #validation_data=val_ds, #X_train, y_train[:len(X_train)], #validation_data=(X_val, y_val), 
                    epochs=100, 
                    callbacks=[early_stop])

Epoch 1/100
  2/249 [..............................] - ETA: 4:01:32 - loss: 8.6897 - accuracy: 0.0000e+00

In [ ]:
model.save('first_simple_model')